# Base Imports

In [8]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import os

from keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Rescaling, GlobalAveragePooling2D, Dense

# Load Data

In [2]:
BATCH_SIZE = 128
RES_ESPACIAL = 32
CHANNELS = 3
NUM_CLASSES = 46

def load_data(path,batch_size,res=RES_ESPACIAL):
  return tf.keras.utils.image_dataset_from_directory(
    path,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(res,res),
    shuffle=True)

In [4]:
train_images = load_data("train_images_cropped/",batch_size=BATCH_SIZE, res=RES_ESPACIAL)
val_images = load_data("val_images_cropped/",batch_size=BATCH_SIZE, res=RES_ESPACIAL)

Found 185101 files belonging to 46 classes.
Found 46276 files belonging to 46 classes.


# Load Metadata

In [5]:
train_metadata = pd.read_csv("metadata_train_df.csv")
val_metadata = pd.read_csv("metadata_val_df.csv")
test_metadata = pd.read_csv("metadata_test_df.csv")

# Model - Categorias imagenes

### Entrenar modelo nuevo

In [9]:
image_raw = keras.Input(shape=(RES_ESPACIAL,RES_ESPACIAL,CHANNELS))
image_scaled = Rescaling(scale=1/255.0)(image_raw)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(image_scaled)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(visual)
visual = MaxPooling2D(pool_size=(2,2))(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = MaxPooling2D(pool_size=(2,2))(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = MaxPooling2D(pool_size=(2,2))(visual)
visual = Conv2D(128,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(128,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(128,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(128,(3,3),activation='relu',padding='same')(visual)
features = GlobalAveragePooling2D()(visual)
hidden =  Dense(256, activation='relu')(features)
outputs = Dense(NUM_CLASSES, activation='softmax')(hidden)

model = tf.keras.Model(inputs=image_raw, outputs=outputs) 
#model.summary()

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        9248  

##### 10 mins aprox

In [11]:
NUM_EPOCHS = 10

optimizer = Adam()
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history_combinado = model.fit(train_images, epochs=NUM_EPOCHS, validation_data=val_images)

Epoch 1/10
1447/1447 [==============================] - 1169s 807ms/step - loss: 2.4579 - accuracy: 0.3038 - val_loss: 2.1746 - val_accuracy: 0.3692
Epoch 2/10
1447/1447 [==============================] - 1116s 770ms/step - loss: 2.0256 - accuracy: 0.4041 - val_loss: 1.9411 - val_accuracy: 0.4243
Epoch 3/10
1447/1447 [==============================] - 1113s 768ms/step - loss: 1.8474 - accuracy: 0.4483 - val_loss: 1.7929 - val_accuracy: 0.4643
Epoch 4/10
1447/1447 [==============================] - 1114s 769ms/step - loss: 1.7530 - accuracy: 0.4748 - val_loss: 1.7546 - val_accuracy: 0.4745
Epoch 5/10
1447/1447 [==============================] - 1116s 770ms/step - loss: 1.6928 - accuracy: 0.4927 - val_loss: 1.7094 - val_accuracy: 0.4863
Epoch 6/10
1447/1447 [==============================] - 1114s 769ms/step - loss: 1.6470 - accuracy: 0.5053 - val_loss: 1.6804 - val_accuracy: 0.4968
Epoch 7/10
1447/1447 [==============================] - 1110s 766ms/step - loss: 1.6068 - accuracy: 0.5168

### Guardar modelo

In [13]:
model_guardar = tf.keras.Model(inputs=image_raw, outputs=outputs)
#model_guardar.summary()
model_guardar.save('Modelos/Categorias 3H sin corte.h5')

### Usar modelo pre-entrenado

In [73]:
model_entrenado = tf.keras.models.load_model('Modelos/Modelo Categorias.h5')
model_entrenado.trainable = False

hidden = tf.keras.layers.Dense(64, activation='relu')(model_entrenado.output)
output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(hidden)
model = tf.keras.Model(inputs=model_entrenado.input, outputs=output)
#model.summary()

# Model - Atributos

### Pre-procesar datos

In [28]:
def extract_metadata(filename):
    attributes_list = []
    corte_list = []
    f = pd.read_csv(filename)
    for _, row in f.iterrows():
        line = row['attribute_labels']
        list_aux = line.split(" ")
        list_aux = list(filter(None, list_aux))
        list_aux = list(map(int, list_aux))
        attributes_list.append(np.array(list_aux))

        x1 = row['x_1']
        y1 = row['y_1']
        x2 = row['x_2']
        y2 = row['y_2']
        corte_list.append(np.array([x1,y1,x2,y2]))
    return np.array(attributes_list), np.array(corte_list)

train_att_list , train_cuadrados_list = extract_metadata("metadata_reduced_train_df_sorted.csv")
val_att_list , val_cuadrados_list = extract_metadata("metadata_val_df_sorted.csv")

print("shape de atributos de entrenamiento: ", train_att_list.shape)
print("shape de atributos de validacion: ", val_att_list.shape)

shape de atributos de entrenamiento:  (37020, 1000)
shape de atributos de validacion:  (46276, 1000)


In [29]:
BATCH_SIZE = 128
RES_ESPACIAL = 64
CHANNELS = 3
NUM_CLASSES = 1000

train_images_folder = 'reduced_train_images_no_class/'
val_images_folder = 'val_images_no_class/'

def create_dataset(images_folder, cuadrados_list, sample_size=100):
    images = os.listdir(images_folder)
    sorted_images = sorted(images, key=lambda x: int(x.split(".")[0].split("/")[-1].split("train_item")[1]))

    image_list = []
    i = 0
    nonCroppedImg = 0

    for image in sorted_images[:sample_size]:
        cuadrado = cuadrados_list[i]
        image_path = images_folder+image
        #print(image_path)
        image = tf.keras.utils.load_img(image_path)
        input_arr = tf.keras.utils.img_to_array(image)
        #print(input_arr.shape, cuadrado[1],cuadrado[3],cuadrado[0],cuadrado[2])
        cropped_img = input_arr[cuadrado[1]:cuadrado[3], cuadrado[0]:cuadrado[2]]
        #print(cropped_img.shape, cuadrado[1],cuadrado[3],cuadrado[0],cuadrado[2])
        cropped_img = np.array([cropped_img])  # Convert single image to a batch.
        if cropped_img.shape[1] == 0 or cropped_img.shape[2] == 0:
            resized_image = np.array(tf.image.resize(np.array([input_arr]), [RES_ESPACIAL, RES_ESPACIAL])[0])
            image_list.append(resized_image)
            nonCroppedImg += 1
        else:
            resized_image = np.array(tf.image.resize(cropped_img, [RES_ESPACIAL, RES_ESPACIAL])[0])
            image_list.append(resized_image)
        i += 1
    print("Se han extraido imagenes de", images_folder, "con exito")
    print("El dataset es de dimensiones:", np.array(image_list).shape)
    print("Imagenes no cortadas: ", nonCroppedImg)

    return np.array(image_list)

train_images = create_dataset(train_images_folder, train_cuadrados_list)
val_images = create_dataset(val_images_folder, val_cuadrados_list)

Se han extraido imagenes de reduced_train_images_no_class/ con exito
El dataset es de dimensiones: (100, 64, 64, 3)
Imagenes no cortadas:  0
Se han extraido imagenes de val_images_no_class/ con exito
El dataset es de dimensiones: (100, 64, 64, 3)
Imagenes no cortadas:  0


### Entrenar modelo nuevo

In [ ]:
image_raw = keras.Input(shape=(RES_ESPACIAL,RES_ESPACIAL,CHANNELS))
image_scaled = Rescaling(scale=1/255.0)(image_raw)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(image_scaled)
visual = Conv2D(32,(3,3),activation='relu',padding='same')(visual)
visual = MaxPooling2D(pool_size=(2,2))(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = Conv2D(64,(3,3),activation='relu',padding='same')(visual)
visual = MaxPooling2D(pool_size=(2,2))(visual)
visual = Conv2D(128,(3,3),activation='relu',padding='same')(visual)
features = GlobalAveragePooling2D()(visual)
hidden =  Dense(256, activation='relu')(features)
outputs = Dense(NUM_CLASSES, activation='sigmoid')(hidden)

model = tf.keras.Model(inputs=image_raw, outputs=outputs) 
#model.summary()

In [32]:
NUM_EPOCHS = 1

optimizer = Adam()
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history_atributos = model.fit(train_images, train_att_list[:len(train_images)], validation_data=[val_images, val_att_list[:len(val_images)]], epochs=NUM_EPOCHS)

157/157 [==============================] - 31s 193ms/step - loss: 0.0194 - accuracy: 0.0134


### Guardar modelo

In [ ]:
model_guardar = tf.keras.Model(inputs=image_raw, outputs=outputs)
model_guardar.summary()
model_guardar.save('Modelo atributos.h5')

## Usar modelo pre-entrenado

In [36]:
model_entrenado = tf.keras.models.load_model('MODELO_ATRIBUTOS_1EPOCH_10000SAMPLESIZE.h5')
model_entrenado.trainable = False

hidden = tf.keras.layers.Dense(64, activation='relu')(model_entrenado.output)
output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(hidden)
model = tf.keras.Model(inputs=model_entrenado.input, outputs=output)
#model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_10 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 32, 32, 64)        18496     
                                                           